In [87]:
import requests
import time
from bs4 import BeautifulSoup
import csv


In [56]:
def extract(jobtype,page=1):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'}
    r = requests.get(url=f'https://builtin.com/jobs/remote?search={jobtype}&page={page}',headers=headers)
    if r.status_code == 200:
        return r.content
    else:
        print(f'status code {r.status_code} sorry :(')
    return None

In [57]:
#extract job info from job cards and return a list of dictionarys with the title description salary date posted employee count and years experience.
def pull_data(soup):
    data = []
    job_cards = soup.find_all('div', {'data-id': 'job-card'})
    for card in job_cards:

        job_info = {'title':'','description':'','salary':'','date_posted':'','employee_count':'','experience':''}
        job_info['title'] = card.find('div',{'data-id': 'company-title'}).text
        job_info['description'] = card.find('h2').text
        job_info['date_posted'] = card.find('span',string=lambda x: any(substring in x for substring in ['Ago','Yesterday'])).text

        pay_info = card.find('span', string=lambda x: 'Annually' in x)
        if pay_info:
            job_info['salary'] = pay_info.text

        employee_count = card.find('span', string=lambda x: 'Employees' in x)
        if employee_count:
            job_info['employee_count'] = int(employee_count.text.replace(',','').split(' ')[0])
        
        years_experience = card.find('span', string=lambda x: 'Experience' in x)
        if years_experience:
            job_info['experience'] = years_experience.text

        data.append(job_info)
    return data

In [91]:
#find max page number for that jobtype change jobtype to search different types of jobs
jobtype = 'data'
html = extract(jobtype=jobtype)
soup = BeautifulSoup(html, 'html.parser')
max_page = soup.find_all('li', class_='mx-xs page-item')[-1].text

In [61]:
#pull data from all pages
data = []
for i in range(1,int(max_page)+1):
    html = extract(jobtype,i)
    soup = BeautifulSoup(html, 'html.parser')
    data.append(pull_data(soup))
    time.sleep(1)

In [89]:
#get rid of duplicates and merge lists to one
finalData = []
for page in data:
    for item in page:
        if item not in finalData:
            finalData.append(item)

In [90]:
headers = finalData[0].keys()
with open('builtInData.csv', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=headers)
    writer.writeheader()
    writer.writerows(finalData)